# OPTICS CLUSTERING METHOD

In [3]:
#IMPORTS
import yfinance as yf
import pandas as pd
import pandas_ta_classic as ta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import OPTICS
from statsmodels.tsa.stattools import coint, grangercausalitytests
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Semiconductor Stocks List
semiconductor_stocks = [
    # Megacap Leaders & Generalists
    "NVDA", "TSM", "AVGO", "AMD", "INTC", "MU", "TXN", "QCOM", "ADI", "MCHP",
    
    # Equipment & Manufacturing (Picks and Shovels)
    "ASML", "AMAT", "LRCX", "KLAC", "TER", "ENTG", "ONT", "NVMI", "TOELY",
    
    # Specialized (Automotive, Power, Connectivity)
    "ON", "NXPI", "STM", "WOLF", "LSCC", "MPWR", "QRVO", "SWKS", "ALAB", "CRDO",
    
    # Intellectual Property & Design Software
    "ARM", "SNPS", "CDNS", "CEVA",
    
    # Memory & Storage
    "MU", "WDC", "STX",
    
    # Emerging & Mid-Cap
    "GFS", "MRVL", "MTSI", "POWI", "SMTC", "VICR", "CAMT"
]